In [1]:
# Setup the Jupyter version of Dash and import required libraries
from pymongo import MongoClient
from bson.objectid import ObjectId
from dash import Dash, html, dcc, Input, Output, State, dash_table
import dash_leaflet as dl
import pandas as pd
from jupyter_dash import JupyterDash
import plotly.express as px
import base64
import os
import numpy as np
import matplotlib.pyplot as plt

# Import the CRUD module for MongoDB interactions
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Database credentials (replace with secure input method in production)
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)

# Load data from MongoDB using the CRUD module
# Note: sending an empty query returns all records
df = pd.DataFrame.from_records(shelter.read({}))

# Remove the '_id' field to prevent compatibility issues with Dash DataTable
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################

app = JupyterDash(__name__)

# Encode the Grazioso Salvare logo image to base64 for display
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Layout definition for the dashboard UI
app.layout = html.Div([
    html.Center(html.B(html.H1('Michael Cary - SNHU CS-340 Dashboard'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'}),
    html.Hr(),

    # Rescue type filter radio buttons
    html.Label('Rescue Type:'),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset'
    ),

    html.Hr(),

    # Main data table for displaying animal information
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'minWidth': '100px', 'width': '150px', 'maxWidth': '180px'
        }
    ),

    html.Br(),
    html.Hr(),

    # Layout for charts: pie chart and geolocation map side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to filter data based on selected rescue type
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'water':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'mountain':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        query = {}

    filtered_df = pd.DataFrame.from_records(shelter.read(query))
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    return filtered_df.to_dict('records')

# Callback to update pie chart based on filtered data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return []  # Prevents rendering errors when no data is available

    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Breed Distribution')
        )
    ]

# Callback to highlight selected column in DataTable
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []

    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback to update the map location based on selected row from DataTable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or index is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row]['name'])
                ])
            ])
        ])
    ]

# Run the Dash app in a new browser tab or window
app.run_server(mode='external')


Dash app running on http://127.0.0.1:16794/
